In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import logging
from github import Github
import time

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Конфигурация
GITHUB_PAT = "..."  # Замените на ваш GitHub PAT
REPO_NAME = "..."
NOTES_PATH = "..."  # Путь к папке с заметками
CACHE_TIMEOUT = 86400  # Время жизни кэша (сутки)

# Глобальные переменные для кэширования
notes_cache = None
last_fetch_time = 0

def fetch_notes():
    """
    Загружает заметки из репозитория GitHub. Использует кэширование для уменьшения количества запросов.
    
    :return: Список заметок
    """
    global notes_cache, last_fetch_time

    # Если данные в кэше и не устарели, возвращаем их
    if notes_cache and time.time() - last_fetch_time < CACHE_TIMEOUT:
        logger.info("Используем кэшированные заметки.")
        return notes_cache

    logger.info("Загрузка заметок из GitHub...")
    
    try:
        # Авторизация через PAT
        g = Github(GITHUB_PAT)
        repo = g.get_repo(REPO_NAME)
        
        # Получение содержимого папки с заметками
        contents = repo.get_contents(NOTES_PATH)
        notes = []
        while contents:
            content = contents.pop(0)
            if content.type == 'file' and content.name.endswith('.md'):
                notes.append(content.decoded_content.decode('utf-8'))
            elif content.type == 'dir':
                contents.extend(repo.get_contents(content.path))
        
        # Обновляем кэш
        notes_cache = notes
        last_fetch_time = time.time()
        
        logger.info(f"Заметки успешно загружены. Загружено {len(notes)} заметок.")
        return notes
    except Exception as e:
        logger.error(f"Ошибка при загрузке заметок: {e}")
        return []

# Загрузка заметок
notes = fetch_notes()

c:\Users\fivas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-10 16:34:10,967 - INFO - Загрузка заметок из GitHub...
2025-01-10 16:35:29,990 - INFO - Заметки успешно загружены. Загружено 127 заметок.


In [62]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import logging
from sklearn.metrics.pairwise import cosine_similarity
import torch

logger = logging.getLogger(__name__)

# Загрузка модели
logger.info("Загрузка модели для векторизации текста...")
try:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)
    logger.info(f"Модель успешно загружена на устройство: {device}.")
except Exception as e:
    logger.error(f"Ошибка при загрузке модели: {e}")
    raise

# Векторизация заметок
logger.info("Векторизация текста...")
try:
    note_embeddings = model.encode(notes, batch_size=32, show_progress_bar=True)
    logger.info(f"Заметки успешно векторизованы. Размерность эмбеддингов: {note_embeddings.shape}")
except Exception as e:
    logger.error(f"Ошибка при векторизации текста: {e}")
    raise

# Нормализация эмбеддингов
logger.info("Нормализация эмбеддингов...")
try:
    note_embeddings = note_embeddings / np.linalg.norm(note_embeddings, axis=1, keepdims=True)
    logger.info("Эмбеддинги успешно нормализованы.")
except Exception as e:
    logger.error(f"Ошибка при нормализации эмбеддингов: {e}")
    raise

# Создание индекса FAISS
logger.info("Создание индекса FAISS...")
try:
    dimension = note_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # IndexFlatIP для косинусного расстояния
    index.add(note_embeddings)
    logger.info(f"Индекс FAISS создан и заполнен. Размерность индекса: {dimension}")
except Exception as e:
    logger.error(f"Ошибка при создании индекса FAISS: {e}")
    raise

# Проверка качества эмбеддингов
logger.info("Проверка качества эмбеддингов...")
try:
    for i in range(10):
        for j in range(i + 1, 10):
            similarity = cosine_similarity([note_embeddings[i]], [note_embeddings[j]])
            logger.info(f"Косинусное сходство между заметкой {i} и {j}: {similarity[0][0]:.4f}")
except Exception as e:
    logger.error(f"Ошибка при проверке качества эмбеддингов: {e}")

2025-01-10 17:24:21,368 - INFO - Загрузка модели для векторизации текста...
2025-01-10 17:24:21,372 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2025-01-10 17:24:24,304 - INFO - Модель успешно загружена на устройство: cpu.
2025-01-10 17:24:24,305 - INFO - Векторизация текста...
Batches: 100%|██████████| 4/4 [00:24<00:00,  6.07s/it]
2025-01-10 17:24:48,750 - INFO - Заметки успешно векторизованы. Размерность эмбеддингов: (127, 768)
2025-01-10 17:24:48,751 - INFO - Нормализация эмбеддингов...
2025-01-10 17:24:48,754 - INFO - Эмбеддинги успешно нормализованы.
2025-01-10 17:24:48,756 - INFO - Создание индекса FAISS...
2025-01-10 17:24:48,758 - INFO - Индекс FAISS создан и заполнен. Размерность индекса: 768
2025-01-10 17:24:48,759 - INFO - Проверка качества эмбеддингов...
2025-01-10 17:24:48,762 - INFO - Косинусное сходство между заметкой 0 и 1: 0.3610
2025-01-10 17:24:48,768 - INFO - Косинусное сходство между заметкой 0 и 2: 0.2922
2025-01-10 17:24:48,

In [66]:
def find_related_context(query, k=10, threshold=0.4):  # Увеличили k до 10
    """
    Поиск связанных заметок по запросу.

    :param query: Текстовый запрос
    :param k: Количество ближайших заметок для возврата
    :param threshold: Порог релевантности (чем больше, тем строже фильтрация)
    :return: Список кортежей (заметка, расстояние)
    """
    logger.info(f"Поиск связанного контекста для запроса: '{query}'")
    
    try:
        # Векторизация запроса
        query_embedding = model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)  # Нормализация
        logger.info(f"Эмбеддинг запроса: {query_embedding}")
        
        # Поиск ближайших заметок
        distances, indices = index.search(query_embedding.reshape(1, -1), k)
        logger.info(f"Расстояния: {distances}")
        logger.info(f"Индексы: {indices}")
        
        # Фильтрация по порогу релевантности
        related_notes = []
        for i, distance in zip(indices[0], distances[0]):
            if distance > threshold:  # Косинусное расстояние (чем больше, тем лучше)
                related_notes.append((notes[i], distance))  # Сохраняем заметку и её релевантность
            logger.info(f"Заметка {i+1}: расстояние = {distance:.4f}, текст: {notes[i][:100]}...")
        
        logger.info(f"Найдено {len(related_notes)} связанных заметок (с порогом релевантности {threshold}).")
        
        if not related_notes:
            logger.warning("Не найдено заметок, превышающих порог релевантности. Попробуйте уменьшить порог.")
            return []
        
        # Сортировка по релевантности (по убыванию)
        related_notes.sort(key=lambda x: x[1], reverse=True)
        
        return related_notes  # Возвращаем список кортежей (заметка, расстояние)
    except Exception as e:
        logger.error(f"Ошибка при поиске связанного контекста: {e}")
        return []

In [65]:
query = "Как удалить дубликаты в SQL?"
related_notes = find_related_context(query, k=10, threshold=0.35)

if not related_notes:
    print("Не найдено подходящих заметок.")
else:
    for note, distance in related_notes:
        print(f"Заметка: {note[:100]}... (расстояние: {distance:.4f})")

2025-01-10 17:29:04,125 - INFO - Поиск связанного контекста для запроса: 'Как удалить дубликаты в SQL?'
Batches: 100%|██████████| 1/1 [00:00<00:00, 11.17it/s]
2025-01-10 17:29:04,234 - INFO - Эмбеддинг запроса: [[ 8.08423944e-03  8.12986046e-02 -3.59514751e-03  6.37662783e-02
   1.22495564e-02  2.44061910e-02 -2.14163996e-02  4.58693653e-02
   1.33264437e-02  1.07977930e-02 -1.48784872e-02  5.82658947e-02
  -3.24992026e-04  2.64806300e-02 -4.82795238e-02 -6.80076471e-03
   4.48045582e-02 -1.31066013e-02 -1.28967734e-02 -1.19701745e-02
   1.37021742e-03  7.09033245e-03 -1.87009554e-02 -4.15945798e-02
  -2.54753642e-02 -3.74273174e-02  1.61916129e-02  5.72659411e-02
   2.28933003e-02  1.83833595e-02 -3.34800184e-02  5.18775955e-02
  -7.14689959e-03 -2.51467358e-02  1.44783193e-02 -1.70061216e-02
  -1.17907468e-02 -1.55923478e-02 -2.79193278e-02  4.42538001e-02
  -4.26493697e-02  4.20715846e-02 -3.77068110e-02  3.14091779e-02
  -2.79011913e-02  9.62847471e-03 -9.52665582e-02 -1.14418203e-

Заметка: - **Устранить все зависимости**. Прежде чем удалять объект из базы данных, нужно избавиться от огран... (расстояние: 0.6739)
Заметка: #### **DROP TABLE**

Эта команда удаляет таблицу из базы данных, вместе со всеми данными, которые в ... (расстояние: 0.6088)
Заметка: DELETE FROM sales
WHERE id NOT IN (
    SELECT MIN(id)
    FROM sales
    GROUP BY date, product_id,... (расстояние: 0.5601)
Заметка: ### 1. **Основы SQL (DDL и DML)**

- **CREATE, DROP, ALTER**: Научись создавать, удалять и изменять ... (расстояние: 0.5425)
Заметка: ### 1. DROP

- **Что делает**: Удаляет целую таблицу или базу данных из базы данных.
- **Как работае... (расстояние: 0.5409)
Заметка: Триггеры в SQL — **это специальные хранимые процедуры, которые автоматически выполняются при возникн... (расстояние: 0.5103)
Заметка: Приведение данных к 1НФ является важным этапом проектирования хранилищ данных, поскольку тотальные и... (расстояние: 0.4435)
Заметка: `CAST` в SQL — это оператор, который позволяет преобр

In [67]:
from mistralai import Mistral
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Укажите ваш API-ключ Mistral AI
api_key = "..."
client = Mistral(api_key=api_key)

# Функция для генерации ответа с использованием Mistral
def generate_response(query, related_notes):
    """
    Генерация ответа на основе связанного контекста.

    :param query: Вопрос пользователя
    :param related_notes: Список связанных заметок (контекст)
    :return: Ответ от Mistral AI
    """
    try:
        # Проверяем, что related_notes не пуст
        if not related_notes:
            return "Не найдено подходящих заметок для ответа."
        
        # Объединение заметок в контекст
        context = "\n\n".join(related_notes)
        
        # Логируем начало контекста для отладки
        # logger.info(f"Контекст для запроса:\n{context[:500]}...")  # Первые 500 символов контекста

        # Формирование запроса к Mistral
        response = client.chat.complete(
            model="mistral-small-latest",
            messages=[
                {
                    "role": "system",
                    "content": (
                        '''Ты — личный помощник, который отвечает на вопросы, используя только предоставленный контекст. 
                        Ты должен внимательно анализировать контекст и находить в нём всю необходимую информацию для ответа. 
                        Если в контексте есть примеры, таблицы или структурированные данные, используй их для ответа. 
                        Если в контексте нет информации для ответа, скажи, что не знаешь.'''
                    )
                },
                {
                    "role": "user",
                    "content": f"Контекст:\n{context}\n\nВопрос: {query}"
                }
            ],
            max_tokens=30000,  # Ограничение на длину ответа
            temperature=0.1  # Параметр для контроля креативности
        )
        
        # Возвращаем ответ
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Ошибка при запросе к Mistral AI: {str(e)}", exc_info=True)  # Логируем полное сообщение об ошибке
        return "Произошла ошибка при обработке запроса. Пожалуйста, попробуйте позже."

# Пример использования
# query = "Основные отличия хранилища данных от базы данных"
# related_notes = find_related_context(query)  # Используем функцию поиска контекста
# response = generate_response(query, related_notes)
# logger.info(f"Ответ Mistral:\n{response}")

In [68]:
if __name__ == "__main__":
    query = "Расскажи про удаление дубликатов в SQL"
    
    # Поиск связанных заметок
    related_notes_with_distance = find_related_context(query, threshold=0.5)  # Уменьшили порог
    
    if not related_notes_with_distance:
        print("Не найдено подходящих заметок для ответа.")
    else:
        # Извлекаем только тексты заметок
        related_notes = [note[0] for note in related_notes_with_distance]  # Оставляем только тексты
        
        # Генерация ответа с использованием Mistral
        response = generate_response(query, related_notes)
        
        # Вывод ответа
        print(response)

2025-01-10 17:31:21,082 - INFO - Поиск связанного контекста для запроса: 'Расскажи про удаление дубликатов в SQL'
Batches: 100%|██████████| 1/1 [00:00<00:00, 11.98it/s]
2025-01-10 17:31:21,182 - INFO - Эмбеддинг запроса: [[ 0.00392421  0.08022402 -0.00567319  0.06069166  0.00679954  0.01801545
  -0.00703367  0.02842852  0.0155909   0.0020567   0.01986893  0.05561417
   0.00171099  0.03299932 -0.04670826 -0.01536623  0.04991684 -0.00670583
  -0.03014787 -0.01421462  0.01360619 -0.0001747  -0.00234119 -0.04323389
  -0.01796954 -0.03846094  0.01244379  0.06237613  0.02402656  0.03193076
  -0.0131853   0.03740891 -0.00512062 -0.03099046  0.01454784 -0.01739068
  -0.01696935 -0.00647513 -0.05354787  0.04513912 -0.01899676  0.05739245
  -0.01494096  0.02154368 -0.03988779  0.01753413 -0.07746305 -0.0178405
   0.01598407 -0.01027626  0.00370586  0.05793214 -0.0467321   0.01805298
  -0.02107063 -0.05312272  0.00279424  0.07078598  0.00772516 -0.02452724
   0.04322435 -0.00693654 -0.02218919  0

Удаление дубликатов в SQL — это процесс, направленный на очистку таблицы от записей, которые имеют одинаковые значения в одном или нескольких столбцах. Существует несколько подходов к удалению дубликатов, каждый из которых имеет свои преимущества и недостатки. Рассмотрим основные методы:

### 1. Удаление дубликатов с помощью `DELETE` и `NOT IN`

Этот метод использует подзапрос для выбора уникальных записей и затем удаляет все остальные.

```sql
DELETE FROM sales
WHERE id NOT IN (
    SELECT MIN(id)
    FROM sales
    GROUP BY date, product_id, customer_id
);
```

### 2. Удаление дубликатов с помощью `WITH` и `ROW_NUMBER`

Этот метод использует оконные функции для нумерации строк в каждой группе и затем удаляет все строки, кроме первой.

```sql
WITH duplicates AS (
    SELECT id,
           ROW_NUMBER() OVER (PARTITION BY date, product_id, customer_id ORDER BY id) AS rn
    FROM sales
)
DELETE FROM sales
WHERE id IN (SELECT id FROM duplicates WHERE rn > 1);
```

### 3. Изменение структу

In [6]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

In [7]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [9]:
TELEGRAM_BOT_TOKEN = "..."

In [52]:
# Обработчик команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Привет! Я ваш помощник с Mistral AI. Задайте мне вопрос, и я постараюсь помочь.")

# Обработчик текстовых сообщений
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text  # Получаем текст сообщения от пользователя
    logger.info(f"Пользователь спросил: {user_message}")
    
    # Получаем связанные заметки (список кортежей)
    related_notes_with_distance = find_related_context(user_message)
    
    # Извлекаем только тексты заметок и ограничиваем количество
    related_notes = [note[0] for note in related_notes_with_distance[:5]]  # Оставляем только первые 5 заметок
    
    # Генерация ответа с использованием Mistral AI
    response = generate_response(user_message, related_notes)  # Передаем оба аргумента
    
    # Отправка ответа пользователю
    await update.message.reply_text(response)

# Обработчик ошибок
async def error(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.error(f"Ошибка: {context.error}")
    await update.message.reply_text("Произошла ошибка. Пожалуйста, попробуйте ещё раз.")

In [ ]:
# Создаем приложение Telegram-бота
application = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

# Регистрируем обработчики команд и сообщений
application.add_handler(CommandHandler("start", start))
application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

# Регистрируем обработчик ошибок
application.add_error_handler(error)

# Запускаем бота
logger.info("Бот запущен...")
application.run_polling()

2025-01-10 17:32:08,011 - INFO - Бот запущен...
2025-01-10 17:32:08,377 - INFO - HTTP Request: POST https://api.telegram.org/bot7665363916:AAFK_QPnY7C-FYNO7eoUg92n0EI6zQlxDHs/getMe "HTTP/1.1 200 OK"
2025-01-10 17:32:08,480 - INFO - HTTP Request: POST https://api.telegram.org/bot7665363916:AAFK_QPnY7C-FYNO7eoUg92n0EI6zQlxDHs/deleteWebhook "HTTP/1.1 200 OK"
2025-01-10 17:32:08,481 - INFO - Application started
2025-01-10 17:32:18,822 - INFO - HTTP Request: POST https://api.telegram.org/bot7665363916:AAFK_QPnY7C-FYNO7eoUg92n0EI6zQlxDHs/getUpdates "HTTP/1.1 200 OK"
2025-01-10 17:32:18,926 - INFO - HTTP Request: POST https://api.telegram.org/bot7665363916:AAFK_QPnY7C-FYNO7eoUg92n0EI6zQlxDHs/getUpdates "HTTP/1.1 200 OK"
2025-01-10 17:32:18,931 - INFO - Пользователь спросил: Как удалить дубликаты в SQL?
2025-01-10 17:32:18,932 - INFO - Поиск связанного контекста для запроса: 'Как удалить дубликаты в SQL?'
Batches: 100%|██████████| 1/1 [00:00<00:00, 10.64it/s]
2025-01-10 17:32:19,044 - INFO - Э